This Notebook gives an example for comparing fluxes for a primary and secondary objective.

In [4]:

#REQUIRED PACKAGES

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import subprocess
import sys
import re
import os
import warnings
import pickle
from scipy.optimize import least_squares
from IPython.core.display import display
from IPython.core.debugger import set_trace
pd.set_option('display.max_columns', None,'display.max_rows', None)

warnings.filterwarnings('ignore')

from importlib import reload



In [5]:
# Import the neccessary modules

Module_path = '../Modules'  #specify path to modules folder, change as needed for environment


sys.path.insert(0,Module_path)

import max_entropy_functions as me

import base_functions as baf
reload(baf)
import solution_analysis_utils as sa
reload(sa)

<module 'solution_analysis_utils' from '/Users/king339/Projects/Predictive_Phenomics/Code/Bolt/01_Glycolosis_Example/../Modules/solution_analysis_utils.py'>

## IMPORT A MODEL

In [9]:

model_file = 'glycolosis_tca_ppp_prot4.pkl'

#Load model


#Load the model file
S_active,active_reactions, metabolites, Keq= baf.load_model(model_file)

#Load and preprocess model
S, Keq, f_log_counts, vcount_upper_bound = baf.numpy_model_preprocessing(model_file)

## SET THE PRIMARY OBJECTIVE

In [16]:
#for this example will simply maximize flux through all reactions equally

n_react = S.shape[0]
primary_obj_coefs = np.ones(n_react)


## SET THE SECONDARY OBJECTIVE

In [17]:
# In this case secondary objective will be flux through PFK reaction

target_rxn_index = np.where(S_active.index =='PFK' )[0][0]

secondary_obj_coefs = np.zeros(n_react)
secondary_obj_coefs[target_rxn_index] = 1

## SOLVE FOR PRIMARY OBJECTIVE

In [18]:
import flux_opt as Fopt

Fopt = reload(Fopt)


y_p, alpha_p, n_p = Fopt.flux_ent_opt(primary_obj_coefs,vcount_upper_bound,f_log_counts,S,Keq)


Ipopt 3.14.4: max_iter=10000
max_cpu_time=800000
tol=1e-07
acceptable_tol=1e-06
linear_solver=ma57
hsllib=/opt/coinhsl/lib/libcoinhsl.dylib


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:      296
Number of nonzeros in inequality constraint Jacobian.:      286
Number of nonzeros in Lagrangian Hessian.............:       58

Input Error: Incorrect refinement scheme.
Total number of variables............................:      145
                     variables with only lower bounds:        0
                variables with lower and uppe

## SEARCH FOR SOLUTION WITH RESPECT TO SECONDARY OBJECTIVE

given a solution to the primary objective with value: p_obj_value

Find a solution to maximize a secondary objective while ensuring the solution is greater than p_obj_value - tol with respect to the primary objective

Assume that fluxes cannot change direction

In [24]:
import max_alt_flux as maf
maf = reload(maf)



primary_obj_tolerance = 1e1




y_s, alpha_s, n_s = maf.Max_alt_flux(n_p,y_p,vcount_upper_bound, f_log_counts, S, Keq,primary_obj_coefs,primary_obj_tolerance,secondary_obj_coefs)




Ipopt 3.14.4: max_iter=10000
max_cpu_time=800000
tol=1e-07
acceptable_tol=1e-06
linear_solver=ma57
hsllib=/opt/coinhsl/lib/libcoinhsl.dylib


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:      238
Number of nonzeros in inequality constraint Jacobian.:      228
Number of nonzeros in Lagrangian Hessian.............:       29

Input Error: Incorrect refinement scheme.
Total number of variables............................:      116
                     variables with only lower bounds:        0
                variables with lower and uppe

#### Compare Objective Values

In [25]:


og_obj_val = np.sum(y_p*primary_obj_coefs)
print('Original Objective Value')
print(og_obj_val)



new_og_obj_val = np.sum(y_s*primary_obj_coefs)
print('New Objective Value')
print(new_og_obj_val)

Original Objective Value
373.4735822180213
New Objective Value
363.47357879237626


### COMPARE Secondary Objective Values

In [26]:

secondary_obj_val = np.sum(y_p*secondary_obj_coefs)
print('Original secondary value')
print(secondary_obj_val)



new_secondary_obj_val = np.sum(y_s*secondary_obj_coefs)
print('New secondary value')
print(new_secondary_obj_val)

Original secondary value
1.1020449845158138
New secondary value
1.536810863323876


## COMPARISON OF THE CHANGES IN FLUX

In [28]:
Flux_comparison = pd.DataFrame([])
Flux_comparison['Reaction'] = S_active.index
Flux_comparison['Original flux'] = y_p
Flux_comparison['New flux'] = y_s
Flux_comparison['Relative Difference'] = (y_s - y_p)/np.abs(y_p)

Flux_comparison.iloc[(-np.abs(Flux_comparison['Relative Difference'].values)).argsort()]


,Reaction,Original flux,New flux,Relative Difference
10,TPI,1.102045e+00,1.536811e+00,3.945083e-01
11,FBA,1.102045e+00,1.536811e+00,3.945083e-01
17,PFK,1.102045e+00,1.536811e+00,3.945083e-01
16,PGI,-2.199465e+01,-1.982079e+01,9.883570e-02
25,TKT1,1.154835e+01,1.067880e+01,-7.529608e-02
27,TKT2,1.154835e+01,1.067880e+01,-7.529608e-02
20,G6PDH2r,3.464504e+01,3.203640e+01,-7.529608e-02
21,PGL,3.464504e+01,3.203640e+01,-7.529608e-02
23,RPE,2.309669e+01,2.135760e+01,-7.529608e-02
22,GND,3.464504e+01,3.203640e+01,-7.529608e-02
